In [1]:
# This file contains tests draft for the Field. It is incomplete.
# It contains only tests which were used during development and which are useful.
import os
import sys
import numpy as np

sys.path.append('../..') # for running py-script
sys.path.append('../../..') # for running this notebook directly

from seismiqb import Field
from seismiqb.src.geometry import export

In [5]:
SEED = 10
SAVING_DIR = './'
DATESTAMP = "555"
CUBE_PATH = 'tmp.sgy'
CUBE_SHAPE=(10, 10, 10)

In [6]:
%%time
# Create fake cube
rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

with open(os.path.join(SAVING_DIR, f'tmp/test_array_{DATESTAMP}.npy'), 'wb') as outfile:
    np.save(outfile, data_array)
    
export.make_segy_from_array(array=data_array, path_segy=CUBE_PATH, zip_segy=False, sample_rate=2., delay=50, pbar='t')

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 38.5 ms, sys: 0 ns, total: 38.5 ms
Wall time: 36.4 ms


In [7]:
%%time
# Init a field
field = Field(geometry=CUBE_PATH)

CPU times: user 32.8 ms, sys: 3.03 ms, total: 35.8 ms
Wall time: 33.9 ms


In [80]:
# Test dump_matrix (and dump_charisma)
# Create and dump a random matrix
matrix = rng.integers(low=0, high=10, size=CUBE_SHAPE[:2])

path=os.path.join(SAVING_DIR, f'tmp/test_matrix_{DATESTAMP}')

field.dump_matrix(matrix=matrix, path=path)

# Open and transform saved data
df = pd.read_csv(path, sep=r'\s+', header=None)

opened_points = df.values
opened_points = field.lines_to_cubic(opened_points)
opened_points = np.round(opened_points).astype(int)

# Create matrix from points array
opened_matrix = np.empty_like(matrix)
opened_matrix[opened_points[:, 0], opened_points[:, 1]] = opened_points[:, 2]

assert np.array_equal(matrix, opened_matrix)